In [1]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 140 kB 48.2 MB/s 
     |████████████████████████████████| 180 kB 47.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=ee548dd1b6b1b31eb03cef24ecb831c04a85c0f70c08168cc4cf4afef95cf9ef
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=0cdcf94eeacecfd39db04cf4b7d1e36432bceef5deb7669d11c2df132a915435
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
import wandb
from tqdm import tqdm

In [3]:
wandb.init(project="17_flowers")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [5]:

model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
in_features = model.fc.in_features
model.fc = nn.Linear(in_features,17)

In [7]:
model = model.to(device)

In [8]:
ct = 0
for child in model.children():
  ct +=1
  if ct < 45:
     for param in child.parameters():
         param.requires_grad =True

In [9]:
config = wandb.config
config.learning_rate = 0.01
config.batch_size =32
config.epochs = 15

In [10]:
transform = transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(20),
                                transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

In [11]:
dataset = torchvision.datasets.ImageFolder(root="/content/drive/MyDrive/Flowers-20210921T131033Z-001/Flowers/Test", transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset,batch_size=config.batch_size, shuffle=True,num_workers=2)

In [12]:
optimizer = torch.optim.Adam(model.parameters(),lr=config.learning_rate)
loss_function = torch.nn.CrossEntropyLoss()

In [13]:
def calc_acc(preds,labels):
  _,preds_max = torch.max(preds, 1)
  acc = torch.sum(preds_max==labels.data,dtype = torch.float64)/len(preds)
  return acc

In [15]:
# train
wandb.watch(model)


for epoch in range(config.epochs):
  train_loss = 0.0
  train_acc = 0.0
  for images, labels in tqdm(train_data_loader):

    images, labels = images.to(device), labels.to(device)
    # print(images.shape)
    optimizer.zero_grad()
    # images = images.float()
    preds = model(images)

    loss = loss_function(preds,  labels)
    loss.backward()

    optimizer.step()

    train_loss += loss
    train_acc +=calc_acc(preds, labels)

  total_loss = train_loss/len(train_data_loader)
  total_acc = train_acc/len(train_data_loader)
  if epoch % 2 == 0:
     wandb.log({"loss": total_loss})
     wandb.log({"acc": total_acc})

  # total_acc = train_acc/len(train_data_loader)
  print(f"Epoch:{epoch}: Loss:{total_loss}, Acc:{total_acc}")
  
  print("-------------------------------------")
  

100%|██████████| 9/9 [00:06<00:00,  1.39it/s]


Epoch:0: Loss:3.1194911003112793, Acc:0.0798611111111111
-------------------------------------


100%|██████████| 9/9 [00:06<00:00,  1.42it/s]


Epoch:1: Loss:3.101414918899536, Acc:0.07638888888888888
-------------------------------------


100%|██████████| 9/9 [00:06<00:00,  1.42it/s]


...

Epoch:12: Loss:2.1609714031219482, Acc:0.2638888888888889
-------------------------------------


100%|██████████| 9/9 [00:06<00:00,  1.41it/s]


Epoch:13: Loss:1.9648761749267578, Acc:0.34027777777777773
-------------------------------------


100%|██████████| 9/9 [00:06<00:00,  1.42it/s]

Epoch:14: Loss:1.8898614645004272, Acc:0.3958333333333333
-------------------------------------


In [ ]:
import cv2
img1 = cv2.imread('/content/apple.jpg')
img1

In [ ]:
convert_tensor=transforms.ToTensor()
img2=convert_tensor(img1)
img2